In [1]:
repl.pprinter() = repl.pprinter().copy(defaultHeight = 5 )

# Delta tables

Spark SQL has become a standard way to process data in a distributed way. The dataframe and dataset api allow us to do all kinds of transformations with our data.

Normaly, when we speak about file formats in big data, we mostly use parquet. As we show in the [previous notebook](./Working%20with%20parquet.ipynb), parquet is a columnar file format created for big data precesess. The amount of metadata, and columnar way to storage our data, combined with a great sinergy with Apache spark allows us to try to read only the esencial data. But for this format as we show, and for all formats in big data, we have a problem to administrate this files in a file system point of view. When we need to do a mantainance in this tables, due to cleaning, or GDPR actions, we allways work more or less in the following way.
First, stop everithing related with this file, nobody can read it.
Second, launch the process and write the result in original folder.

Let's start, but first, let's prepare the enviroment, with a reference `wd` to the folder that we will persist the data, create the spark session and we will be ready to rock 🤘.

In [2]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import $ivy.`io.delta::delta-core:0.5.0`
import $ivy.`io.circe::circe-parser:0.12.3`
import $ivy.`com.lihaoyi::os-lib:0.2.7`
import org.apache.spark.sql._
import io.circe.parser._
import java.time.Instant

import $ivy.$                                  

import $ivy.$                           

import $ivy.$                              

import $ivy.$                          

import org.apache.spark.sql._

import io.circe.parser._

import java.time.Instant

In [3]:
val file = "delta-example"

file: String = "delta-example"

In [4]:
val wd = os.pwd / file

wd: os.package.pwd.ThisType = root/'home/'jovyan/'work/"delta-example"

In [5]:
os.remove.all(wd)

In [6]:
val spark = SparkSession.builder().appName("delta-test").master("local[1]").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/04/13 13:09:25 INFO SparkContext: Running Spark version 2.4.5
20/04/13 13:09:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
20/04/13 13:09:25 INFO SparkContext: Submitted application: delta-test
20/04/13 13:09:25 INFO SecurityManager: Changing view acls to: jovyan
20/04/13 13:09:25 INFO SecurityManager: Changing modify acls to: jovyan
20/04/13 13:09:25 INFO SecurityManager: Changing view acls groups to: 
20/04/13 13:09:25 INFO SecurityManager: Changing modify acls groups to: 
20/04/13 13:09:25 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jovyan); groups with view permissions: Set(); users  with modify permissions: Set(jovyan); groups with modify permissions: Set()
20/04/13 13:09:26 INFO Utils: Successfully started service 'sparkDriver' on port 37215.
20/0

spark: SparkSession = org.apache.spark.sql.SparkSession@612c2819

In [7]:
spark.sparkContext.setLogLevel("ERROR")

## Version 0

Now that we have our spark session, we want to try this delta format, so we are going to write some data to it. The only diference to write parquet, is that the format is "delta". (Remember, by default the save mode is "Error if exist")

In [8]:
spark.range(1,10).write.format("delta").save(file)

To be sure, lets show what we have written.

In [9]:
spark.read.format("delta").load(file).show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



Take a look of the folder were we wrote. We will se that we have written a single parquet file, due to the single core of our standalone cluster. But also, it created a folder called `_delta_log`

In [10]:
os.list(wd).foreach(println)

/home/jovyan/work/delta-example/.part-00000-2a1a7ab1-2f5e-4530-bb70-436ba04cc095-c000.snappy.parquet.crc
/home/jovyan/work/delta-example/_delta_log
/home/jovyan/work/delta-example/part-00000-2a1a7ab1-2f5e-4530-bb70-436ba04cc095-c000.snappy.parquet


Listing the files, we see that it contais a json file inside.

In [11]:
val logFolder = wd / "_delta_log"
os.list(logFolder).foreach(println)

/home/jovyan/work/delta-example/_delta_log/00000000000000000000.json


logFolder: wd.ThisType = root/'home/'jovyan/'work/"delta-example"/'_delta_log

Here we see the main difference with parquet. Each time we write something in our delta table, we will have a commit log in this folder. But to learn about it, let's deep dive in them.

We can use [circe](https://circe.github.io/circe/) to see the content of this file in a pretty way.

In [12]:
val version0 = os.list(logFolder).find(_.last.endsWith("0.json")).get
val version0Json = os.read.lines(version0).map(x => parse(x).right.get.spaces2)

version0: os.Path = root/'home/'jovyan/'work/"delta-example"/'_delta_log/"00000000000000000000.json"
version0Json: collection.mutable.WrappedArray[String] = WrappedArray(
  """{
  "commitInfo" : {
    "timestamp" : 1586783385009,
...

Each line of the file is a json document.The first one, has the commit infomation, showing us the timestamp of the commit, the type of operation, with the parameters of it, and also if it was a bling append, an append that didn't take previous commits in account.

In [13]:
println(version0Json(0))

{
  "commitInfo" : {
    "timestamp" : 1586783385009,
    "operation" : "WRITE",
    "operationParameters" : {
      "mode" : "ErrorIfExists",
      "partitionBy" : "[]"
    },
    "isBlindAppend" : true
  }
}


The second document, contains information of the protocol used for delta table, with the minimun version of reader and writer needed.

In [14]:
println(version0Json(1))

{
  "protocol" : {
    "minReaderVersion" : 1,
    "minWriterVersion" : 2
  }
}


The third, contains metadata information, with the schema, the format of the files used, in this case parquet. 

In [15]:
println(version0Json(2))

{
  "metaData" : {
    "id" : "65c12125-d7a9-4270-9c1d-8bee6f83644c",
    "format" : {
      "provider" : "parquet",
      "options" : {
        
      }
    },
    "schemaString" : "{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}",
    "partitionColumns" : [
    ],
    "configuration" : {
      
    },
    "createdTime" : 1586783383392
  }
}


And the last, shows that it has added a new file with the path of the new parquet file created.

In [16]:
println(version0Json(3))

{
  "add" : {
    "path" : "part-00000-2a1a7ab1-2f5e-4530-bb70-436ba04cc095-c000.snappy.parquet",
    "partitionValues" : {
      
    },
    "size" : 483,
    "modificationTime" : 1586783384000,
    "dataChange" : true
  }
}


## Version1

Let's append a new bunch of data, and take a look of what it contains now.

In [17]:
spark.range(10,20).write.format("delta").mode(SaveMode.Append).save(file)
spark.read.format("delta").load(file).show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



Nothing unexpected, it mixed the previous data with the new one. If we check the parquet files, we will haver our previous file, and a new one, as expected.

In [18]:
os.list(wd).filter(_.last.endsWith("parquet")).foreach(println)

/home/jovyan/work/delta-example/part-00000-27a15051-cebb-42d0-93fd-8fa7914b964d-c000.snappy.parquet
/home/jovyan/work/delta-example/part-00000-2a1a7ab1-2f5e-4530-bb70-436ba04cc095-c000.snappy.parquet


And in the log? Take a look.

In [19]:
os.list(logFolder).foreach(println)

/home/jovyan/work/delta-example/_delta_log/00000000000000000000.json
/home/jovyan/work/delta-example/_delta_log/00000000000000000001.json


We can see that there is a new file that ends in 0001.json, this is the latest version of the log, let's take a look

In [20]:
val version1 = os.list(logFolder).find(_.last.endsWith("1.json")).get
val version1Json = os.read.lines(version1).map(x => parse(x).right.get)

version1: os.Path = root/'home/'jovyan/'work/"delta-example"/'_delta_log/"00000000000000000001.json"
version1Json: collection.mutable.WrappedArray[io.circe.Json] = WrappedArray(
  JObject(
    object[commitInfo -> {
  "timestamp" : 1586783397056,
...

This `version 1` file only has two lines, one like the previous one with the information of the commit.

In [21]:
println(version1Json(0))

{
  "commitInfo" : {
    "timestamp" : 1586783397056,
    "operation" : "WRITE",
    "operationParameters" : {
      "mode" : "Append",
      "partitionBy" : "[]"
    },
    "readVersion" : 0,
    "isBlindAppend" : true
  }
}


And the second one, also like the [version 0](#Version-0)

In [22]:
println(version1Json(1))

{
  "add" : {
    "path" : "part-00000-27a15051-cebb-42d0-93fd-8fa7914b964d-c000.snappy.parquet",
    "partitionValues" : {
      
    },
    "size" : 472,
    "modificationTime" : 1586783397000,
    "dataChange" : true
  }
}


You won't see any difference between delta table and workning with parquet at this point.

## Version2

But lets overwrite our data, and we will see the first real difference.

In [23]:
spark.range(20,30).write.format("delta").mode(SaveMode.Overwrite).save(file)

Let's check that the overwrite was done ok.

In [24]:
spark.read.format("delta").load(file).show()

+---+
| id|
+---+
| 20|
| 21|
| 22|
| 23|
| 24|
| 25|
| 26|
| 27|
| 28|
| 29|
+---+



If we list our parquet files, we will see that not only it has added a new one, it also mantains the previous that we created.

In [25]:
os.list(wd).filter(_.last.endsWith("parquet")).foreach(println)

/home/jovyan/work/delta-example/part-00000-27a15051-cebb-42d0-93fd-8fa7914b964d-c000.snappy.parquet
/home/jovyan/work/delta-example/part-00000-2a1a7ab1-2f5e-4530-bb70-436ba04cc095-c000.snappy.parquet
/home/jovyan/work/delta-example/part-00000-300ce4ec-52b7-423e-8e0e-285503ed66bd-c000.snappy.parquet


Why? The anser is in the new log file created, this time with index 2.

In [26]:
val version2 = os.list(logFolder).find(_.last.endsWith("2.json")).get
val version2Json = os.read.lines(version2).map(x => parse(x).right.get.spaces2)

version2: os.Path = root/'home/'jovyan/'work/"delta-example"/'_delta_log/"00000000000000000002.json"
version2Json: collection.mutable.WrappedArray[String] = WrappedArray(
  """{
  "commitInfo" : {
    "timestamp" : 1586783407642,
...

In [27]:
println(version2Json(0))

{
  "commitInfo" : {
    "timestamp" : 1586783407642,
    "operation" : "WRITE",
    "operationParameters" : {
      "mode" : "Overwrite",
      "partitionBy" : "[]"
    },
    "readVersion" : 1,
    "isBlindAppend" : false
  }
}


In [28]:
println(version2Json(1))
println(version2Json(2))
println(version2Json(3))

{
  "add" : {
    "path" : "part-00000-300ce4ec-52b7-423e-8e0e-285503ed66bd-c000.snappy.parquet",
    "partitionValues" : {
      
    },
    "size" : 472,
    "modificationTime" : 1586783406000,
    "dataChange" : true
  }
}
{
  "remove" : {
    "path" : "part-00000-2a1a7ab1-2f5e-4530-bb70-436ba04cc095-c000.snappy.parquet",
    "deletionTimestamp" : 1586783407641,
    "dataChange" : true
  }
}
{
  "remove" : {
    "path" : "part-00000-27a15051-cebb-42d0-93fd-8fa7914b964d-c000.snappy.parquet",
    "deletionTimestamp" : 1586783407641,
    "dataChange" : true
  }
}


Now we see that the log is indicating that its adding a new parquet file (line with "add" key), but also, that in this version, it must not take in account the previous files(lines with the "remove" key).

Now we can uderstand that what delta table is doing. It isn't resolving the actions in the folder like parquet that would have eresed all previous data. What delta is doing it's keeping all the data files, in this case parquet files, and logging the changes in the '_delta_log' throw versioning.

This way, delta can recreate in any moment no only the last version, as we would do in a normal parquet file, also previous state versions.

So if we can recreate them, how we can read the file, but like the version 0 or 1? indicating in the options the version that we want to read.

In [29]:
spark.read.format("delta").option("versionAsOf", 0).load(file).show

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [30]:
spark.read.format("delta").option("versionAsOf", 1).load(file).show

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



Also, we can use the option "timestampAsOf" to get the last version available at that moment. For this we can take advance of the optics implementation in circe JSON object to obtain the timestamp of the version 1.

In [31]:
val version1Timestamp = version1Json(0).hcursor.downField("commitInfo").get[Long]("timestamp").right.get

version1Timestamp: Long = 1586783397056L

And create two timestamps, one slightly earlier to the version 1 and other later to version 1. The format of the date must be 

In [32]:
val previousToVersion1 = Instant.ofEpochMilli(version1Timestamp - 5000)
val laterToVersion1 = Instant.ofEpochMilli(version1Timestamp + 5000)

previousToVersion1: Instant = 2020-04-13T13:09:52.056Z
laterToVersion1: Instant = 2020-04-13T13:10:02.056Z

So if we use a timestamp previous to version 1 (and later thant version 0 of course), we will recover all the data in that moment.

In [33]:
spark.read.format("delta").option("timestampAsOf", previousToVersion1.toString).load(file).show

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



And the same for the later than version 1

In [34]:
spark.read.format("delta").option("timestampAsOf", laterToVersion1.toString).load(file).show

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



So, if it doesn't overwrite the data, could we read and delete from the same delta table? Let's try